## Liiga lühikeste kirjete eemaldamine baasist 

Muudab *collection*'is validated=False kõigi kirjete puhul, mille tekst on lühem kui kaks sõna. 


In [2]:
from qdrant_client import QdrantClient, models
from datetime import datetime, timedelta
import uuid

In [1]:
client_host = 'localhost'
client_port = 6333

collection_name = 'test123'

In [3]:
client = QdrantClient(client_host, port=client_port)

In [4]:
client.get_collection(collection_name).vectors_count

4

In [ ]:
def scroll(collection_name, limit=10000, offset=0):
    points, next_page_offset = client.scroll(
        collection_name=collection_name,
        limit=limit,
        offset=offset)
    return points, next_page_offset

def get_invalid_point_ids(points):
    """Get id-s of entries with less than two words from a given set of Point objects."""
    invalid_point_ids = []
    for point in points:
        point_id = point.id
        text = point.payload['text']

        if text.strip().count(' ') == 0 and text.strip().count('\n') == 0:
            invalid_point_ids.append(point_id)
    return invalid_point_ids

### Alla kahe sõnaliste kannete uuendamine baasis -- valid==False

In [ ]:
limit = 10000
offset=0
points = ['*']*limit

# Iterate over the whole collection.
while offset is not None:
    points, offset = scroll(collection_name, limit=limit, offset=offset)
    invalid_point_ids = get_invalid_point_ids(points)

    # Get invalid points
    invalid_points, _ = client.scroll(
    collection_name=collection_name,
    scroll_filter=models.Filter(
        must=[
            models.HasIdCondition(has_id=invalid_point_ids),
        ],
    ),
    limit=limit
    )

    # Update invalid points with valid = False in the collection

    for point in invalid_points:

        payload = point.payload
        payload['validated'] = False

        client.set_payload(
          collection_name=collection_name,
          payload=payload,
          points=[point.id])


### Kuupäevade lisamine baasi - 

In [ ]:
limit = 1000
offset=0
points = ['*']*limit

# Iterate over the whole collection.
while offset is not None:
    points, offset = scroll(collection_name, limit=limit, offset=offset)

    for point in points:
        payload = point.payload
        payload['date_created'] = (datetime.date(datetime.today()) - timedelta(days=13)).isoformat()
        payload['date_modified'] = (datetime.date(datetime.today()) - timedelta(days=13)).isoformat()

        client.set_payload(
            collection_name=collection_name,
            payload=payload,
            points=[point.id])

In [ ]:
collection_name

---

In [ ]:
import re

def validate_chunk_text(text, min_word_count=10):
    text = re.sub('(\n+| +)', ' ', text).strip()
    if len(text.split(' ')) < min_word_count:
        return False
    return True

In [ ]:
limit = 10000
offset=0
points = ['*']*limit

# Iterate over the whole collection.
while offset is not None:
    points, offset = scroll(collection_name, limit=limit, offset=offset)
    for point in points:
        if not validate_chunk_text(point.payload['text']):
            if point.payload["validated"] is True and point.payload["content_type"] != 'term':
                print(point.payload['filename'])
                print(f'text: {point.payload["text"]}\ntype:{point.payload["content_type"]}\nvalid:{point.payload["validated"]}')
                print('\n', '*'*50, '\n')